# ****1) Importing required libraries****

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

In [ ]:
import pandas as pd
import numpy as np
import keras
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping

import sklearn.metrics as metrics
from keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)

# **2) Importing Data**

**2.1) Original Dataset**

In [ ]:
import os
import pandas as pd

MildDemented_dir1 = r'/kaggle/input/augmented-alzheimer-mri-dataset/OriginalDataset/MildDemented'
ModerateDemented_dir1 = r'/kaggle/input/augmented-alzheimer-mri-dataset/OriginalDataset/ModerateDemented'
NonDemented_dir1 = r'/kaggle/input/augmented-alzheimer-mri-dataset/OriginalDataset/NonDemented'
VeryMildDemented_dir1 = r'/kaggle/input/augmented-alzheimer-mri-dataset/OriginalDataset/VeryMildDemented'

filepaths = []
labels = []
dict_list = [MildDemented_dir1, ModerateDemented_dir1, NonDemented_dir1, VeryMildDemented_dir1]
class_labels = ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very MildDemented']

for i, j in enumerate(dict_list):
    flist = os.listdir(j)
    for f in flist:
        fpath = os.path.join(j, f)
        filepaths.append(fpath)
        labels.append(class_labels[i])

Fseries1 = pd.Series(filepaths, name="filepaths")
Lseries1 = pd.Series(labels, name="labels")
Alzheimer_data1 = pd.concat([Fseries1, Lseries1], axis=1)
Alzheimer_df1 = pd.DataFrame(Alzheimer_data1)
print(Alzheimer_df1.head())
print(Alzheimer_df1["labels"].value_counts())

**2.2)Augmented Dataset**

In [ ]:
import os
import pandas as pd

MildDemented_dir = r'/kaggle/input/augmented-alzheimer-mri-dataset/AugmentedAlzheimerDataset/MildDemented'
ModerateDemented_dir = r'/kaggle/input/augmented-alzheimer-mri-dataset/AugmentedAlzheimerDataset/ModerateDemented'
NonDemented_dir = r'/kaggle/input/augmented-alzheimer-mri-dataset/AugmentedAlzheimerDataset/NonDemented'
VeryMildDemented_dir = r'/kaggle/input/augmented-alzheimer-mri-dataset/AugmentedAlzheimerDataset/VeryMildDemented'

filepaths = []
labels = []
dict_list = [MildDemented_dir, ModerateDemented_dir, NonDemented_dir, VeryMildDemented_dir]
class_labels = ['Mild Demented', 'Moderate Demented', 'Non Demented', 'Very MildDemented']

for i, j in enumerate(dict_list):
    flist = os.listdir(j)
    for f in flist:
        fpath = os.path.join(j, f)
        filepaths.append(fpath)
        labels.append(class_labels[i])

Fseries = pd.Series(filepaths, name="filepaths")
Lseries = pd.Series(labels, name="labels")
Alzheimer_data = pd.concat([Fseries, Lseries], axis=1)
Alzheimer_df = pd.DataFrame(Alzheimer_data)
print(Alzheimer_df.head())
print(Alzheimer_df["labels"].value_counts())

In [ ]:
Alzheimer_df.shape

# **3) Data Split**

In [ ]:
train_images, test_images = train_test_split(Alzheimer_df, test_size=0.3, random_state=42)
train_set, val_set = train_test_split(Alzheimer_df, test_size=0.2, random_state=42)

In [ ]:
print(train_set.shape)
print(test_images.shape)
print(val_set.shape)
print(train_images.shape)

In [ ]:
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)
train = image_gen.flow_from_dataframe(dataframe= train_set,x_col="filepaths",y_col="labels",
                                      target_size=(244,244),
                                      color_mode='rgb',
                                      class_mode="categorical", #used for Sequential Model
                                      batch_size=32,
                                      shuffle=False            #do not shuffle data
                                     )
test = image_gen.flow_from_dataframe(dataframe= test_images,x_col="filepaths", y_col="labels",
                                     target_size=(244,244),
                                     color_mode='rgb',
                                     class_mode="categorical",
                                     batch_size=32,
                                     shuffle= False
                                    )
val = image_gen.flow_from_dataframe(dataframe= val_set,x_col="filepaths", y_col="labels",
                                    target_size=(244,244),
                                    color_mode= 'rgb',
                                    class_mode="categorical",
                                    batch_size=32,
                                    shuffle=False
                                   )

In [ ]:
import numpy as np
from collections import Counter

# Function to count the number of items in each class
def count_classes(data_gen):
    class_labels = list(data_gen.class_indices.keys())
    class_counts = Counter(data_gen.labels)
    return {class_labels[i]: class_counts[i] for i in range(len(class_labels))}

# Count classes in each set
train_class_counts = count_classes(train)
test_class_counts = count_classes(test)
val_class_counts = count_classes(val)

print("Train set class counts:", train_class_counts)
print("Test set class counts:", test_class_counts)
print("Validation set class counts:", val_class_counts)

In [ ]:
classes=list(train.class_indices.keys())
print (classes)

In [ ]:
def show_images(image_gen):
    test_dict = test.class_indices
    classes = list(test_dict.keys())
    images, labels=next(image_gen) # get a sample batch from the generator
    plt.figure(figsize=(20,20))
    length = len(labels)
    if length<25:
        r=length
    else:
        r=25
    for i in range(r):
        plt.subplot(5,5,i+1)
        image=(images[i]+1)/2 #scale images between 0 and 1
        plt.imshow(image)
        index=np.argmax(labels[i])
        class_name=classes[index]
        plt.title(class_name, color="green",fontsize=16)
        plt.axis('off')
    plt.show()

In [ ]:
show_images(train)

# **3) Usefull Functions**

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_curves(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


# **4) Model Creation**

# **4.1)Basic CNN Model**

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=128, kernel_size=(8, 8), strides=(3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3, 3)),
    
    keras.layers.Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1, 1), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1, 1), strides=(1, 1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    keras.layers.Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    
    keras.layers.MaxPool2D(pool_size=(2, 2)),
    
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.optimizers.SGD(learning_rate=0.001),
    metrics=['accuracy']
)
history = model.fit(train, epochs=20, validation_data=val, validation_freq=1)

In [ ]:
plot_loss_curves(history)

In [ ]:
Train_loss, Train_accuracy = model.evaluate(train)

print(f'\nTrain loss: {Train_loss:.2f}')
print(f'Train Accuracy: {Train_accuracy*100:.2f} %')

In [ ]:
valid_loss, valid_accuracy = model.evaluate(val)

print(f'\nValidation loss: {valid_loss:.2f}')
print(f'Validation Accuracy: {valid_accuracy*100:.2f} %')

In [ ]:
loss, accuracy = model.evaluate(test)

print(f'\nTest loss: {loss:.2f} ')
print(f'Test Accuracy: {accuracy*100:.2f} %')

In [ ]:
preds = model.predict(test)
y_pred = np.argmax(preds, axis=1)

In [ ]:
g_dict = test.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= classes, title = 'Confusion Matrix')

# Classification report
print(classification_report(test.classes, y_pred, target_names= classes))


# **4.2)Efficient Net B2 Model**

In [ ]:
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB2

# Build the model with the adjusted input shape
efficientnet_b2_model = Sequential()
efficientnet_b2_model.add(EfficientNetB2(weights='imagenet', include_top=False, input_shape=(244, 244, 3)))
efficientnet_b2_model.add(GlobalAveragePooling2D())
efficientnet_b2_model.add(Dense(256, activation='relu'))
efficientnet_b2_model.add(Dropout(0.5))
efficientnet_b2_model.add(Dense(4, activation='softmax'))

efficientnet_b2_model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)

# Assuming 'train' and 'val' are your training and validation datasets
efficientnet_b2_history = efficientnet_b2_model.fit(
    train,
    epochs=10,
    validation_data=val,
    verbose=1
)


In [ ]:
plot_loss_curves(efficientnet_b2_history)

In [ ]:
Train_loss, Train_accuracy = efficientnet_b2_model.evaluate(train)

print(f'\nTrain loss: {Train_loss:.2f}')
print(f'Train Accuracy: {Train_accuracy*100:.2f} %')

In [ ]:
valid_loss, valid_accuracy = efficientnet_b2_model.evaluate(val)

print(f'\nValidation loss: {valid_loss:.2f}')
print(f'Validation Accuracy: {valid_accuracy*100:.2f} %')

In [ ]:
loss, accuracy =efficientnet_b2_model.evaluate(test)

print(f'\nTest loss: {loss:.2f} ')
print(f'Test Accuracy: {accuracy*100:.2f} %')

In [ ]:
preds = efficientnet_b2_model.predict(test)
y_pred = np.argmax(preds, axis=1)

In [ ]:
g_dict = test.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= classes, title = 'Confusion Matrix')

# Classification report
print(classification_report(test.classes, y_pred, target_names= classes))

# **4.3) ResNet50 Model**

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten

# Build the ResNet50 model with the correct input shape
resnet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(244, 244, 3))
resnet50_model = Sequential()
resnet50_model.add(resnet50_base_model)
resnet50_model.add(Flatten())
resnet50_model.add(Dense(256, activation='relu'))
resnet50_model.add(Dropout(0.5))
resnet50_model.add(Dense(4, activation='softmax'))
resnet50_model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['accuracy']
)

# Train the ResNet50 model
resnet50_history = resnet50_model.fit(
    train,
    epochs=10,
    validation_data=val,
    verbose=1
)


In [ ]:
plot_loss_curves(resnet50_history)

In [ ]:
Train_loss, Train_accuracy = resnet50_model.evaluate(train)

print(f'\nTrain loss: {Train_loss:.2f}')
print(f'Train Accuracy: {Train_accuracy*100:.2f} %')

In [ ]:
valid_loss, valid_accuracy = resnet50_model.evaluate(val)

print(f'\nValidation loss: {valid_loss:.2f}')
print(f'Validation Accuracy: {valid_accuracy*100:.2f} %')

In [ ]:
loss, accuracy = resnet50_model.evaluate(test)

print(f'\nTest loss: {loss:.2f} ')
print(f'Test Accuracy: {accuracy*100:.2f} %')

In [ ]:
preds = resnet50_model.predict(test)
y_pred = np.argmax(preds, axis=1)

In [ ]:
g_dict = test.class_indices
classes = list(g_dict.keys())

# Confusion matrix
cm = confusion_matrix(test.classes, y_pred)
plot_confusion_matrix(cm= cm, classes= classes, title = 'Confusion Matrix')

# Classification report
print(classification_report(test.classes, y_pred, target_names= classes))